В этом ноутбуке представлен пример предсказания, а также развертывание сервиса по генерации аккордов.

In [1]:
import numpy as np
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, AdamW
from tqdm import tqdm
import re
from typing import List, Tuple, Optional, Mapping,  Self, NamedTuple
import pandas as pd
from datasets import load_dataset
import ast
import itertools
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from transformers import BertTokenizer, BertForMaskedLM
from chord_tokenaizer import ChordTokenizer, ChordTokenizerHF
from bert_architecture import Encoder, BERTLM_MLM_And_Genre
warnings.filterwarnings('ignore')

In [2]:
def load_and_clean_dataset():

    dataset = load_dataset("lluccardoner/melodyGPT-song-chords-text-1")
    df = dataset['train'].to_pandas()

    def clean_chords(text):
        if not isinstance(text, str):
            return ""

        text = re.sub(r'\t', ' ', text)
        text = re.sub(r'\b(INTRO|VERSE|CHORUS|BRIDGE|OUTRO|SOLO|PRE-CHORUS)\b', '', text, flags=re.IGNORECASE)
        chords = re.findall(r'[A-G][#b]?(?:m)?(?:1[0-3]|[2-9])?', text)

        return ' '.join(chords)

    df['cleaned_chords'] = df['chords_str'].apply(clean_chords)

    df = df[df['cleaned_chords'].str.len() > 0]

    return df


In [3]:
dataset = load_and_clean_dataset()

In [4]:
list_cleaned_chords = dataset['cleaned_chords'].map(lambda x: x.split(' '))

chords_set = set()
for row in list_cleaned_chords:
  chords_set.update(row)

Загрузим токенайзер

In [5]:
def load_tokenizer(filepath, chord_set):
    """Загружает токенайзер из файла"""
    tokenizer_data = torch.load(filepath)
    

    tokenizer = ChordTokenizer(chord_set)

    tokenizer.vocab = tokenizer_data['vocab']
    tokenizer.reverse_vocab = tokenizer_data['reverse_vocab']
    tokenizer.special_tokens = tokenizer_data['special_tokens']
    
    print(f"Токенайзер загружен из {filepath}")
    return tokenizer

loaded_tokenizer = load_tokenizer('chord_tokenizer.pth', chords_set)

Токенайзер загружен из chord_tokenizer.pth


Загрузим модель

In [6]:
VOCAB_SIZE = len(loaded_tokenizer.vocab)
BATCH_SIZE = 64  
MAX_SEQ_LEN = 50 
N_LAYERS = 4 
EMBEDDING_SIZE = 256  
NUM_HEADS = 8
HEAD_EMBEDDING_SIZE = EMBEDDING_SIZE // NUM_HEADS 
FCCN_HIDDEN_SIZE = 512
DROPOUT = 0.2

n_epoch = 5 

In [7]:
encoder = Encoder(
    vocab_size=VOCAB_SIZE,
    n_layers=N_LAYERS,
    embedding_size=EMBEDDING_SIZE,
    num_heads=NUM_HEADS,
    fcnn_hidden_size=FCCN_HIDDEN_SIZE,
    dropout=DROPOUT
)

In [8]:
def load_model(filepath, encoder_class, encoder_config):

    checkpoint = torch.load(filepath, map_location='cuda:0')
    
    encoder = encoder_class(**encoder_config)
    model = BERTLM_MLM_And_Genre(encoder, num_genres=15)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    
    print(f"Модель загружена из {filepath}")
    
    return model


encoder_config = {
    'n_layers': N_LAYERS,
    'num_heads': NUM_HEADS,
    'fcnn_hidden_size': FCCN_HIDDEN_SIZE,
    'vocab_size': VOCAB_SIZE,
    'embedding_size': EMBEDDING_SIZE,
}

loaded_model = load_model('best_genre_bert.pth', Encoder, encoder_config)

Модель загружена из best_genre_bert.pth


In [6]:
loaded_model.eval()

BERTLM(
  (_encoder): Encoder(
    (_embeddings): BERTEmbedding(
      (_embeddings): Embedding(1213, 64, padding_idx=0)
      (_segment_embeddings): Embedding(3, 64, padding_idx=0)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (_positional_embeddings): RotaryPositionEmbedding()
    (_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (_mha): RoPEMultiHeadedAttention(
          (_positional_embedding): RotaryPositionEmbedding()
          (_Q): Linear(in_features=64, out_features=64, bias=True)
          (_K): Linear(in_features=64, out_features=64, bias=True)
          (_V): Linear(in_features=64, out_features=64, bias=True)
          (_W_proj): Linear(in_features=64, out_features=64, bias=True)
          (_dropout): Dropout(p=0.1, inplace=False)
          (_layernorm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (_fcnn): FCNNBlock(
          (_linear1): Linear(in_features=64, out_features=256, bias=False)
          (_linear2): Linear(in_f

In [34]:
import pickle

with open('cats_dict_mapping.pickle', 'rb') as file:
    cats_dict = pickle.load(file)

reverse_cat_dict = {v:k for k,v in cats_dict.items()}

In [57]:
def predict(model, tokenizer, chord_sequence, return_genre_pred=False):

    model.eval()
    
    chord_sequence = chord_sequence.split()
    mask_token = '[MASK]'
    masked_sequence = [chord if chord != '?' else mask_token for chord in chord_sequence]
    
    if mask_token not in masked_sequence:
        masked_sequence.append(mask_token)

    input_ids = tokenizer.convert_tokens_to_ids(masked_sequence)
    mask_index = input_ids.index(4)
    inputs = torch.tensor(input_ids)

    with torch.no_grad():
        inputs = inputs.unsqueeze(0)
        if return_genre_pred:
            mlm_output, genre_output = model(inputs, return_genre_pred)
             
        else:
            mlm_output = model(inputs)


    masked_predictions = mlm_output[0, mask_index] 

    predicted_index = torch.argmax(masked_predictions).item()
    predicted_chord = tokenizer.reverse_vocab.get(predicted_index, '[UNK]')

    masked_sequence[mask_index] = predicted_chord
    result = " ".join(masked_sequence)

    if return_genre_pred:
        pred_genre = genre_output.argmax(dim=-1).item()
        genre = reverse_cat_dict.get(pred_genre, 'unknown')
        return result, genre
    else:
        return result


In [62]:
result, genre = predict(loaded_model, loaded_tokenizer, 'D ? D F B B F B B F B B F D F C D D C D F', return_genre_pred = True)

In [63]:
result

'D D D F B B F B B F B B F D F C D D C D F'

In [64]:
genre

'rock'

### Поднимаем gradio

In [61]:
import gradio as gr
import numpy as np
import io
import wave
import random


# Синтез гитарного звука 
SAMPLE_RATE = 22050

NOTE_FREQS = {
    'C': 261.6, 'C#': 277.2, 'Db': 277.2,
    'D': 293.7, 'D#': 311.1, 'Eb': 311.1,
    'E': 329.6, 'F': 349.2, 'F#': 370.0,
    'Gb': 370.0, 'G': 392.0, 'G#': 415.3,
    'Ab': 415.3, 'A': 440.0, 'A#': 466.2,
    'Bb': 466.2, 'B': 493.9
}

INTERVALS = {
    'maj': [0, 4, 7],
    'min': [0, 3, 7],
    'dim': [0, 3, 6],
    'aug': [0, 4, 8],
    '7': [0, 4, 7, 10],
    'maj7': [0, 4, 7, 11],
    'm7': [0, 3, 7, 10],
    'sus2': [0, 2, 7],
    'sus4': [0, 5, 7],
    '5': [0, 7]
}

def parse_chord(chord):
    chord = chord.strip()
    root = ''
    quality = ''
    for note in sorted(NOTE_FREQS.keys(), key=lambda x: -len(x)):
        if chord.startswith(note):
            root = note
            quality = chord[len(note):]
            break
    if quality == '':
        quality = 'maj'
    if quality in ['m', 'min']:
        quality = 'min'
    elif quality in ['maj', '']:
        quality = 'maj'
    if quality not in INTERVALS:
        quality = 'maj'
    return root, quality

def chord_to_wave_guitar(chord, duration=1.1):
    root, quality = parse_chord(chord)
    base_freq = NOTE_FREQS.get(root, 261.6)
    intervals = INTERVALS[quality]

    t = np.linspace(0, duration, int(SAMPLE_RATE * duration), False)
    wave_data = np.zeros_like(t)
    for interval in intervals:
        freq = base_freq * 2 ** (interval / 12)
        envelope = np.exp(-3 * t)  # гитарное затухание
        wave_data += 0.3 * np.sin(2 * np.pi * freq * t) * envelope

    wave_data = wave_data / np.max(np.abs(wave_data))
    pcm = (wave_data * 32767).astype(np.int16)

    buf = io.BytesIO()
    with wave.open(buf, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(SAMPLE_RATE)
        wf.writeframes(pcm.tobytes())
    return buf.getvalue()

def sequence_to_audio(chords: str):
    chunks = [chord_to_wave_guitar(ch) for ch in chords.split()]
    full = io.BytesIO()
    with wave.open(full, "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(SAMPLE_RATE)
        for w in chunks:
            wf.writeframes(wave.open(io.BytesIO(w)).readframes(99999999))
    return full.getvalue()


def predict_chords(user_input, predict_genre_flag):
    if predict_genre_flag:
        result_chords, res_genre = predict(loaded_model, loaded_tokenizer, user_input, return_genre_pred=True)
        audio_bytes = sequence_to_audio(result_chords)
    else:
        result_chords = predict(loaded_model, loaded_tokenizer, user_input)
        audio_bytes = sequence_to_audio(result_chords)
        res_genre = 'Жанр не предсказан'
    return result_chords, (SAMPLE_RATE, np.frombuffer(audio_bytes, dtype=np.int16)), res_genre



iface = gr.Interface(
    fn=predict_chords,
    inputs=[
        gr.Textbox(label="Введите аккорды", placeholder="Am ? Dm E"),
        gr.Checkbox(label="Предсказать жанр", value=False)
    ],
    outputs=[
        gr.Textbox(label="Сгенерированные аккорды"),
        gr.Audio(label="Аудио", type="numpy"),
        gr.Textbox(label="Предсказанный жанр")
    ],
    title="🎸 Генератор аккордов", 
    allow_flagging='never'
)
iface.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
